In [12]:
# Imports
import sys
sys.path.append('../../../utils')

from default_imports import *

In [13]:
# Load data
raw_train_df = pd.read_csv('../data/raw/train.csv')
raw_test_df = pd.read_csv('../data/raw/test.csv')

# Data Exploration
01_data_exploration.ipynb

In [14]:
raw_train_df

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058
749996,749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242
749997,749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000
749998,749998,Style Guide,Episode 47,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,100.72939


In [15]:
raw_train_df.isna().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [16]:
from sklearn.feature_selection import mutual_info_regression

X = raw_train_df.copy().drop(columns=['id', 'Guest_Popularity_percentage'])
X['Median_Episode_Length_minutes'] = (X
                         .groupby('Podcast_Name')['Episode_Length_minutes']
                         .transform('median'))
X['Episode_Length_minutes'] = X['Episode_Length_minutes'].fillna(X.pop('Median_Episode_Length_minutes'))
X['Number_of_Ads'] = X['Number_of_Ads'].fillna(0)
y = X.pop('Listening_Time_minutes')

for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

discrete_features = X.dtypes == int

mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
mi_scores

Episode_Length_minutes        2.008034
Host_Popularity_percentage    0.762686
Episode_Title                 0.684701
Podcast_Name                  0.416284
Publication_Day               0.242985
Number_of_Ads                 0.239432
Genre                         0.199516
Publication_Time              0.175106
Episode_Sentiment             0.141512
Name: MI Scores, dtype: float64

In [17]:
X = raw_train_df.copy()[['Listening_Time_minutes', 'Guest_Popularity_percentage']].dropna()
y = X.pop('Listening_Time_minutes')

mi_scores = mutual_info_regression(X, y)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
mi_scores

Guest_Popularity_percentage    0.747924
Name: MI Scores, dtype: float64

## Categorical and Discrete Data

In [18]:
categorical_data = raw_train_df.select_dtypes(include='object')
categorical_data['Number_of_Ads'] = raw_train_df['Number_of_Ads']

categorical_data.nunique()

Podcast_Name          48
Episode_Title        100
Genre                 10
Publication_Day        7
Publication_Time       4
Episode_Sentiment      3
Number_of_Ads         12
dtype: int64

In [19]:
categorical_data.isna().sum()

Podcast_Name         0
Episode_Title        0
Genre                0
Publication_Day      0
Publication_Time     0
Episode_Sentiment    0
Number_of_Ads        1
dtype: int64

In [20]:
categorical_data['Number_of_Ads'] = categorical_data['Number_of_Ads'].fillna(0)

### Episode Title